In [5]:
from mario_scenes.load_data import load_scenes_info
from mario_replays.load_data import load_replay_data
import os
import pandas as pd

In [ ]:
scenes_df = load_scenes_info(format='df')

df_list = []
sourcedata = '../../mario_learning.behavior/sourcedata/'
output_csv = '../outputdata/clips_metadata.csv'
if not os.path.exists(output_csv):    
    for clips_dir in os.listdir(sourcedata):
        if 'replays' not in clips_dir and '.' not in clips_dir: 
            print(clips_dir)
            sidecars_df = load_replay_data(os.path.join(sourcedata, clips_dir), type='metadata')
            
            if "StateClipCode" in sidecars_df.columns:
                sidecars_df = sidecars_df.rename(columns={"StateClipCode": "ClipCode"})
            df_list.append(sidecars_df)

    full_df = pd.concat(df_list)
    full_df.loc[full_df["Model"].isnull(), "Model"] = "human"

    # for Model == 'human', we need a column Phase_stage with the value 'Early' or 'Late'. This value is determined by the ClipCode : if the ClipCode is below cutoff, it is 'Early', otherwise it is 'Late'. The cutoff is the median determined separately for each combination of Model='human', Subject and Phase (so Model='human', Subject=1, Phase='practice' has its own cutoff, etc...)
    mask = full_df["Model"].eq("human")

    # median ClipCode per (Subject, Phase) for human rows
    med = (
        full_df.loc[mask]
            .groupby(["Subject", "Phase", "LevelFullName"])["ClipCode"]
            .transform("median")
    )

    # assign Phase_stage without any Python-level loop
    full_df.loc[mask, "Phase_stage"] = np.where(
        full_df.loc[mask, "ClipCode"] < med,
        "Early",
        "Late"
    )

    full_df.to_csv(output_csv, index=False)
else:
    full_df = pd.read_csv(output_csv)
full_df

ppo_mario_ep-20
ppo_mario_ep-2000


In [ ]:
### Difficulty per pattern


In [ ]:
### Difficulty per scene cluster